In [1]:
import pandas as pd
import getpass
from functions import connect_mysql, mysql_list

# Input User id
try:
    # password input hashed
    id = getpass.getpass("id: ") 
except Exception as ex:
    print('Error Occured : ', ex)
#Input password
try:
    # password input hashed
    pwd = getpass.getpass("password: ") 
except Exception as ex:
    print('Error Occured : ', ex)

#host for RDS connection
host = "mdsi-dsp-spr-2020.cehiwpryiego.ap-southeast-2.rds.amazonaws.com"
mydb = connect_mysql(host, id, pwd)

if mydb: 
    print("Connection Successful")
    

id: ········
password: ········
Connection Successful


In [2]:
from functions import mysql_list
mydb = connect_mysql(host, id, pwd)
mycursor = mydb.cursor()
mycursor.execute("SHOW DATABASES;")
# create a list of database
db_list = mysql_list(mycursor.fetchall())
print(db_list) 
# asx is the database to be used
db = "asx"

['asx', 'information_schema', 'mysql', 'performance_schema']


In [3]:
mydb = connect_mysql(host, id, pwd, db)
mycursor = mydb.cursor()
mycursor.execute("SHOW TABLES;")
tb_list = mysql_list(mycursor.fetchall())
print(tb_list)

['asx200', 'asx_all', 'cashrate']


In [4]:
from IPython.display import display, HTML
mydb = connect_mysql(host, id, pwd, db)

for tb in tb_list:
    df = pd.read_sql_query("SELECT * FROM %s" % (tb), con=mydb)
    print(tb)
    display(df.head())

mydb.close()

asx200


,Code,Company,Trading Halt
0,A2M,The a2 Milk Company Ltd,N
1,ABC,Adbri Ltd,N
2,ABP,Abacus Property Group,N
3,AGL,AGL Energy Ltd,N
4,AIA,Auckland International Airport Ltd,N


asx_all


,Code,Company name,GICS industry group
0,14D,1414 DEGREES LIMITED,Capital Goods
1,1AD,ADALTA LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences"
2,1AG,ALTERRA LIMITED,Commercial & Professional Services
3,1ST,1ST GROUP LIMITED,Health Care Equipment & Services
4,2BE,TUBI LIMITED,Energy


cashrate


,Date,Change,CashRate
0,1990-01-23,-0.50 to -1.00,17.00 to 17.50
1,1990-02-15,-0.5,16.50 to 17.00
2,1990-03-07,0,16.50 to 17.00
3,1990-04-04,-1.00 to -1.50,15.00 to 15.50
4,1990-05-02,0,15.00 to 15.50
